In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
!pip install transformers torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-6lk2jkka
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-6lk2jkka
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=355c6a20fceeb59d67a28cdd0da096102deb33154d017733598ef3c0514bf2b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-dguavh27/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms.functional import vflip
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import clip
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torchvision.transforms import ToTensor
from tqdm import tqdm
import torchvision.utils as vutils

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
cifar100_dataset = datasets.CIFAR100(
    root="./data", train=False, download=True,
    transform=transforms.ToTensor()
)

100%|██████████| 169M/169M [00:13<00:00, 12.6MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [ ]:
def grayscale(image):
    return transforms.Grayscale(num_output_channels=3)(image)

def upside_down(image):
    return vflip(image)

def blurry(image):
    return image.filter(ImageFilter.GaussianBlur(radius=3))

def zoomed(image, scale=1.5):
    w, h = image.size
    crop_size = int(min(w, h) / scale)
    return image.crop(((w - crop_size) // 2, (h - crop_size) // 2, (w + crop_size) // 2, (h + crop_size) // 2)).resize((w, h))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])

#Training

## Plain training

In [ ]:
class CIFAR100_Clean(Dataset):
    def __init__(self, cifar100_dataset):
        self.cifar100 = cifar100_dataset
        self.to_tensor = transforms.Compose([ToTensor(), normalize])

    def __len__(self):
        return len(self.cifar100)

    def __getitem__(self, idx):
        image, label = self.cifar100[idx]
        label_name = self.cifar100.classes[label]
        prompt = f"An image of {label_name}."
        image = self.to_tensor(image)
        image = torch.clamp(image, 0, 1)

        return image, prompt

# Initialize dataset and dataloader
cifar100 = datasets.CIFAR100(root="./data", train=True, download=True)
clean_dataset = CIFAR100_Clean(cifar100)
dataloader = DataLoader(clean_dataset, batch_size=32, shuffle=True)


# Load CLIP model and processor
model_clean = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Define optimizer and training parameters
optimizer = AdamW(model_clean.parameters(), lr=1e-4)
epochs = 4


# Training Loop
for epoch in range(epochs):
    model_clean.train()
    total_loss = 0

    for images, prompts in tqdm(dataloader):
        images = images.to(device)
        inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_normalize=False).to(device)

        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(logits_per_image.size(0)).to(device)
        loss_image = F.cross_entropy(logits_per_image, labels)
        loss_text = F.cross_entropy(logits_per_text, labels)
        loss = (loss_image + loss_text) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


Files already downloaded and verified


100%|██████████| 1563/1563 [04:02<00:00,  6.46it/s]


Epoch 1/4, Loss: 2.5360


100%|██████████| 1563/1563 [04:02<00:00,  6.46it/s]


Epoch 2/4, Loss: 2.5388


100%|██████████| 1563/1563 [04:02<00:00,  6.45it/s]


Epoch 3/4, Loss: 2.5394


100%|██████████| 1563/1563 [04:03<00:00,  6.42it/s]

Epoch 4/4, Loss: 2.5392


## Use random augmentations

In [ ]:
class CIFAR100Augmented(Dataset):
    def __init__(self, cifar100_dataset):
        self.cifar100 = cifar100_dataset
        self.augmentations = {
            "grayscale": transforms.Grayscale(num_output_channels=3),
            "blurry": transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            "zoomed": transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.CenterCrop((32, 32)),
                transforms.Resize((32, 32))
            ])
        }
        self.to_tensor = transforms.Compose([ToTensor(), normalize])

    def __len__(self):
        return len(self.cifar100)

    def __getitem__(self, idx):
        image, label = self.cifar100[idx]
        label_name = self.cifar100.classes[label]
        prompt = f"An image of {label_name}."

        # Apply random augmentation
        augmentation_key = torch.randint(0, len(self.augmentations), (1,)).item()
        augmentation_name = list(self.augmentations.keys())[augmentation_key]
        augmentation = self.augmentations[augmentation_name]

        augmented_image = augmentation(image)
        augmented_image = self.to_tensor(augmented_image)
        augmented_image = torch.clamp(augmented_image, 0, 1)
        augmented_prompt = f"A {augmentation_name} image of {label_name}."

        return augmented_image, augmented_prompt

cifar100 = datasets.CIFAR100(root="./data", train=True, download=True)
augmented_dataset = CIFAR100Augmented(cifar100)
dataloader = DataLoader(augmented_dataset, batch_size=32, shuffle=True)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

optimizer = AdamW(model.parameters(), lr=1e-4)
epochs = 4


for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, prompts in tqdm(dataloader):
        images = images.to(device)
        inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_normalize=False).to(device)

        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(logits_per_image.size(0)).to(device)
        loss_image = F.cross_entropy(logits_per_image, labels)
        loss_text = F.cross_entropy(logits_per_text, labels)
        loss = (loss_image + loss_text) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


Files already downloaded and verified


100%|██████████| 1563/1563 [04:38<00:00,  5.61it/s]


Epoch 1/4, Loss: 3.1284


100%|██████████| 1563/1563 [04:37<00:00,  5.64it/s]


Epoch 2/4, Loss: 2.5506


100%|██████████| 1563/1563 [04:37<00:00,  5.64it/s]


Epoch 3/4, Loss: 2.3395


100%|██████████| 1563/1563 [04:36<00:00,  5.66it/s]

Epoch 4/4, Loss: 2.1912


In [ ]:
model.save_pretrained("/content/drive/MyDrive/COSE474/fine_tuned_clip_rand_aug")

In [ ]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")

## Using only gray-scale

In [ ]:
model_gray = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

class CIFAR100Augmented_Gray(Dataset):
    def __init__(self, cifar100_dataset):
        self.cifar100 = cifar100_dataset
        self.augmentations = {
            "grayscale": transforms.Grayscale(num_output_channels=3),
            "blurry": transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            "zoomed": transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.CenterCrop((32, 32)),
                transforms.Resize((32, 32))
            ])
        }
        self.to_tensor = transforms.Compose([ToTensor(), normalize])

    def __len__(self):
        return len(self.cifar100)

    def __getitem__(self, idx):
        image, label = self.cifar100[idx]
        label_name = self.cifar100.classes[label]
        prompt = f"An image of {label_name}."

        augmentation_key = 0
        augmentation_name = list(self.augmentations.keys())[augmentation_key]
        augmentation = self.augmentations[augmentation_name]

        augmented_image = augmentation(image)
        augmented_image = self.to_tensor(augmented_image)
        augmented_image = torch.clamp(augmented_image, 0, 1)
        augmented_prompt = f"A {augmentation_name} image of {label_name}."

        return augmented_image, augmented_prompt

cifar100 = datasets.CIFAR100(root="./data", train=True, download=True)
augmented_dataset = CIFAR100Augmented_Gray(cifar100)
dataloader = DataLoader(augmented_dataset, batch_size=32, shuffle=True)


model_gray = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


optimizer = AdamW(model_gray.parameters(), lr=1e-4)
epochs = 4


for epoch in range(epochs):
    model_gray.train()
    total_loss = 0

    for images, prompts in tqdm(dataloader):
        images = images.to(device)
        inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_normalize=False).to(device)

        outputs = model_gray(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(logits_per_image.size(0)).to(device)
        loss_image = F.cross_entropy(logits_per_image, labels)
        loss_text = F.cross_entropy(logits_per_text, labels)
        loss = (loss_image + loss_text) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

Files already downloaded and verified


100%|██████████| 1563/1563 [04:19<00:00,  6.02it/s]


Epoch 1/4, Loss: 3.4661


100%|██████████| 1563/1563 [04:18<00:00,  6.04it/s]


Epoch 2/4, Loss: 3.4653


100%|██████████| 1563/1563 [04:19<00:00,  6.02it/s]


Epoch 3/4, Loss: 3.4658


100%|██████████| 1563/1563 [04:19<00:00,  6.03it/s]

Epoch 4/4, Loss: 3.4653


## Using only blur

In [ ]:
model_blur = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

class CIFAR100Augmented_Blur(Dataset):
    def __init__(self, cifar100_dataset):
        self.cifar100 = cifar100_dataset
        self.augmentations = {
            "grayscale": transforms.Grayscale(num_output_channels=3),
            "blurry": transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            "zoomed": transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.CenterCrop((32, 32)),
                transforms.Resize((32, 32))
            ])
        }
        self.to_tensor = transforms.Compose([ToTensor(), normalize])

    def __len__(self):
        return len(self.cifar100)

    def __getitem__(self, idx):
        image, label = self.cifar100[idx]
        label_name = self.cifar100.classes[label]
        prompt = f"An image of {label_name}."

        augmentation_key = 1
        augmentation_name = list(self.augmentations.keys())[augmentation_key]
        augmentation = self.augmentations[augmentation_name]

        augmented_image = augmentation(image)
        augmented_image = self.to_tensor(augmented_image)
        augmented_image = torch.clamp(augmented_image, 0, 1)
        augmented_prompt = f"A {augmentation_name} image of {label_name}."

        return augmented_image, augmented_prompt


cifar100 = datasets.CIFAR100(root="./data", train=True, download=True)
augmented_dataset = CIFAR100Augmented_Blur(cifar100)
dataloader = DataLoader(augmented_dataset, batch_size=32, shuffle=True)


model_blur = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


optimizer = AdamW(model_blur.parameters(), lr=1e-4)
epochs = 4


for epoch in range(epochs):
    model_blur.train()
    total_loss = 0

    for images, prompts in tqdm(dataloader):
        images = images.to(device)
        inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_normalize=False).to(device)

        outputs = model_blur(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(logits_per_image.size(0)).to(device)
        loss_image = F.cross_entropy(logits_per_image, labels)
        loss_text = F.cross_entropy(logits_per_text, labels)
        loss = (loss_image + loss_text) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

Files already downloaded and verified


100%|██████████| 1563/1563 [04:59<00:00,  5.21it/s]


Epoch 1/4, Loss: 3.4661


100%|██████████| 1563/1563 [05:02<00:00,  5.17it/s]


Epoch 2/4, Loss: 3.4653


100%|██████████| 1563/1563 [05:00<00:00,  5.20it/s]


Epoch 3/4, Loss: 3.4653


100%|██████████| 1563/1563 [05:00<00:00,  5.21it/s]

Epoch 4/4, Loss: 3.4655


## Using only zooming

In [ ]:
model_zoom = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

class CIFAR100Augmented_Zoom(Dataset):
    def __init__(self, cifar100_dataset):
        self.cifar100 = cifar100_dataset
        self.augmentations = {
            "grayscale": transforms.Grayscale(num_output_channels=3),
            "blurry": transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            "zoomed": transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.CenterCrop((32, 32)),
                transforms.Resize((32, 32))
            ])
        }
        self.to_tensor = transforms.Compose([ToTensor(), normalize])

    def __len__(self):
        return len(self.cifar100)

    def __getitem__(self, idx):
        image, label = self.cifar100[idx]
        label_name = self.cifar100.classes[label]
        prompt = f"An image of {label_name}."

        augmentation_key = 2
        augmentation_name = list(self.augmentations.keys())[augmentation_key]
        augmentation = self.augmentations[augmentation_name]

        augmented_image = augmentation(image)
        augmented_image = self.to_tensor(augmented_image)
        augmented_image = torch.clamp(augmented_image, 0, 1)
        augmented_prompt = f"A {augmentation_name} image of {label_name}."

        return augmented_image, augmented_prompt


cifar100 = datasets.CIFAR100(root="./data", train=True, download=True)
augmented_dataset = CIFAR100Augmented_Zoom(cifar100)
dataloader = DataLoader(augmented_dataset, batch_size=32, shuffle=True)


model_zoom = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


optimizer = AdamW(model_zoom.parameters(), lr=1e-4)
epochs = 4


for epoch in range(epochs):
    model_zoom.train()
    total_loss = 0

    for images, prompts in tqdm(dataloader):
        images = images.to(device)
        inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_normalize=False).to(device)

        outputs = model_zoom(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        labels = torch.arange(logits_per_image.size(0)).to(device)
        loss_image = F.cross_entropy(logits_per_image, labels)
        loss_text = F.cross_entropy(logits_per_text, labels)
        loss = (loss_image + loss_text) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

Files already downloaded and verified


100%|██████████| 1563/1563 [04:47<00:00,  5.45it/s]


Epoch 1/4, Loss: 3.4663


100%|██████████| 1563/1563 [04:45<00:00,  5.47it/s]


Epoch 2/4, Loss: 3.4654


100%|██████████| 1563/1563 [04:45<00:00,  5.48it/s]


Epoch 3/4, Loss: 3.4654


100%|██████████| 1563/1563 [04:45<00:00,  5.47it/s]

Epoch 4/4, Loss: 3.4959
